In [ ]:
import cvxpy as cp
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import scipy.io
# mosek only needed if we don't use MW
#import mosek
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import cProfile
#from baselines import *
from scipy.linalg import sqrtm

In [ ]:
#synthetic example with N datapoints, in d dimensions, with k actions, random regressors
#returns a dictionary of covariates x, labels y, regressors ell.  
def synthetic_example(N,d,k):
    #covariates
    x = np.random.normal(0,1,(N,d))
    #regressors
    ell = np.random.normal(0,1,(k,d))
    #labels
    y = np.zeros((N,k))
    for i in range(N):
        for j in range(k):
            feature = x[i,:]
            regressor = ell[j,:]
            y[i,j] = np.inner(feature,regressor) + np.random.normal(0,1)
    data = {'cov': x, 'label': y, 'reg': ell}
    return data

from scipy.special import logit, expit

def logistic_synthetic_example(N,d,k,corrupt=False):
    #covariates, scaling the covariates to be of large norm amplifies the effect of corruptions
    x = np.random.normal(0,1,(N,d))
    #regressors
    ell = np.random.normal(0,1,(k,d))
    ell = ell/np.linalg.norm(ell)
    #labels
    y = np.zeros((N,k))
    prob_list = np.zeros((N,k))
    for i in range(N):
        for j in range(k):
            feature = x[i,:]
            regressor = ell[j,:]
            prob = expit(np.inner(feature,regressor))
            prob_list[i,j] = prob
            y[i,j] = np.random.binomial(1,prob)
            #deterministic labels
            #if prob > 0.5:
            #    y[i,j] = 1
            #else:
            #    y[i,j] = 0
    corr_frac = 0.1
    corr = int(corr_frac*N)
    print('number of corruptions')
    print(corr)
    if corrupt:
        select = np.zeros((corr,k))
        for j in range(k):
            order = np.argsort(prob_list[:,j])
            select[:,j] = order[:corr]
        for i in range(corr):
            for j in range(k):
                index = int(select[i,j])
                y[index,j] = 1 - y[index,j]
    data = {'cov': x, 'label': y, 'reg': ell}
    return data

#data = synthetic_example(1000,100,10)    
#print(data)

import pandas as pd, numpy as np, re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.datasets import load_svmlight_file

def parse_data(filename):
    with open(filename, "rb") as f:
        infoline = f.readline()
        infoline = re.sub(r"^b'", "", str(infoline))
        n_features = int(re.sub(r"^\d+\s(\d+)\s\d+.*$", r"\1", infoline))
        features, labels = load_svmlight_file(f, n_features=n_features, multilabel=True)
    mlb = MultiLabelBinarizer()
    labels = mlb.fit_transform(labels)
    features = np.array(features.todense())
    features = np.ascontiguousarray(features)
    return features, labels


data_mode = 'logistic_synth'
#data_mode = 'synthetic'
#data_mode = 'real'
N = 1000
d = 100
k = 2

if data_mode == 'logistic_synth':
    #data = logistic_synthetic_example(N,d,k,corrupt=True)
    data = logistic_synthetic_example(N,d,k,corrupt=False)
if data_mode == 'synthetic':
    data = synthetic_example(1000,10,10)    
elif data_mode == 'real': 
    x, y = parse_data("Bibtex_data.txt")
    print(x.shape)
    print(y.shape)
    data = {'cov': x, 'label': y}

x = data['cov']
y = data['label']

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', random_state=0, fit_intercept ='False')
#a = np.zeros((2,3)).reshape(-1,1)
#b = np.ones(2)

#a = np.zeros(10,2).reshape(-1, 1)
#b = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
lab = y[:,0]
model.fit(x, lab)
print(model.coef_)
print(data['reg'])
print('score')
print(model.score(x,lab))



In [ ]:
#logistic regression experiments
#implement the weighted logistic regression
import tensorflow as tf

#sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
#var = tf.Variable(2.5)

#def func(x):
#    return x**2

#cost = lambda(func)(3)

#for _ in range(100):
#    sgd.minimize(cost, var_list=[var])

#var.numpy()
#cost().numpy()
from tensorflow import keras
from tensorflow.keras import layers

#model = keras.Sequential()
#model.add(layers.Dense(64, kernel_initializer='uniform', input_shape=(10,)))
#model.add(layers.Activation('softmax'))

#loss_function = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#model.compile(loss=loss_function, optimizer='adam')

#def custom_loss_function(y_true, y_pred):
#   squared_difference = tf.square(y_true - y_pred)
#   return tf.reduce_mean(squared_difference, axis=-1)

#model.compile(optimizer='adam', loss=custom_loss_function)

#y_true = [12, 20, 29., 60.]
#y_pred = [14., 18., 27., 55.]
#cl = custom_loss_function(np.array(y_true),np.array(y_pred))
#cl.numpy()

x_train = data['cov']
y_train = y[:,0]

#x_val = np.zeros((5,4))
#y_val = np.array([0,0,1,1,1])

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer

ANN_model = Sequential()
ANN_model.add(InputLayer(input_shape=(d, )))
# No hidden layers
ANN_model.add(Dense(1, activation='sigmoid'))
ANN_model.summary()


weights = tf.ones(d)

def custom_loss_function(y_true, y_pred):
    arg1 = y_true
    arg2 = tf.math.log(y_pred)
    arg3 = tf.subtract(tf.ones(d),y_true)
    arg4 = tf.subtract(tf.ones(d),y_pred)
    inp1 = tf.multiply(arg1,arg2)
    inp2 = tf.multiply(arg3,arg4)
    loss_vec = tf.math.scalar_mul(-1.0,tf.add(inp1,inp2))
    weight_loss_vec = tf.multiply(weights,loss_vec)
    output = tf.reduce_mean(weight_loss_vec, axis=-1)
    #t = tf.square(tf.subtract(y_true,y_pred))
    #output = tf.reduce_mean(t,axis=-1)

    return output

opt=tf.keras.optimizers.Adam(learning_rate=0.01)
#loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)
#ANN_model.compile(optimizer=opt,
#                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
#                  metrics=['accuracy'])

ANN_model.compile(optimizer=opt,
                  loss=custom_loss_function,
                  metrics=['accuracy'])


history = ANN_model.fit(x_train, y_train, 
                        epochs=20, batch_size=32,
                        validation_split=0.2, 
                        shuffle=True)

In [ ]:
from mw2 import MW_no_alt_min, get_weights, altmin_step

def isotropic(Xs,fake=False):
    if fake:
        return Xs, np.eye(Xs.shape[1])
    Sig = np.matmul(Xs.T,Xs)
    Sig_sqrt = np.linalg.inv(sqrtm(Sig))
    new_Xs = np.matmul(Xs,Sig_sqrt)
    return new_Xs, Sig_sqrt

def scram(x,y,params):
    AM_steps = params[0]
    altmin_params = params[1:]
    N,d = x.shape
    w = [0.]*d
    a = [1.]*N
    #iso_x, Sig_sqrt = isotropic(x)
    iso_x = x
    Sig_sqrt = np.eye(d)
    AM_steps = 1
    for i in range(AM_steps):
        #print('AM step: ',i)
        w,a = altmin_step(iso_x,y,a,altmin_params,init=False)
    final_w = np.matmul(Sig_sqrt,w)    
    return final_w

#regression oracle 
def regression_oracle(x,y,mode='ols'):
    if mode == 'ols':
        model = LinearRegression()
        model.fit(x,y)
        regressor = model.coef_
    if mode == 'scram':
        lr = 0.5
        lam = 0.2
        MW_steps = 10
        eta = 0.1
        AM_steps = 10
        params = (AM_steps,lr,lam,MW_steps,eta)
        regressor = scram(x,y,params)
    return regressor


reg = regression_oracle(x,y[:,0])

def optimal_reward(y):
    a,b = y.shape
    cumsum = 0
    for i in range(a):
        cumsum = cumsum + np.amax(y[i,:])
    return cumsum/a

max_reward = optimal_reward(y)
print('Optimal Reward')
print(max_reward)

In [ ]:
#contextual bandits takes covariates and labels
def contextual_bandit(cov_label):
    cov = cov_label['cov']
    labels = cov_label['label']
    (N,d) = cov.shape
    (N,k) = labels.shape
    estimators = np.zeros((k,d))
    action_list = []
    mu = k
    delta = 0.1
    gamma = np.sqrt(k*N/(d*np.log(N/d) + 1./(2*delta)))
    params = (mu,gamma)
    rewards = []
    mean_reward = []
    for i in range(N):
        print('iteration: ',i)
        covariate = cov[i,:]
        values = np.zeros(k)
        for j in range(k):
            est = estimators[j,:]
            values[j] = np.inner(est,covariate)
        action = select_action(values,params)
        action_list.append(action)
        bandit_feedback = labels[i,action]
        rewards.append(bandit_feedback)
        (data_x,data_y) = get_data(cov_label,action_list,action) 
        
        #bug, ols can run on one datapoint but scram can't
        #estimators[action,:] = regression_oracle(data_x,data_y,mode='ols')
        estimators[action,:] = regression_oracle(data_x,data_y,mode='scram')
        print('action')
        print(action)
        print('average reward')
        #print(rewards)
        avg_reward = sum(rewards)/len(rewards)
        mean_reward.append(avg_reward)
        print(avg_reward)
    return mean_reward

def get_data(cov_label,action_list,action):
    cov = cov_label['cov']
    labels = cov_label['label']
    (N,d) = cov.shape
    (N,k) = labels.shape
    length = len(action_list)
    count = 0
    for i in range(length):
        if action_list[i] == action:
            count = count + 1
    
    data_x = np.zeros((count,d))
    data_y = np.zeros(count)
    counter = 0
    for i in range(len(action_list)):
        if action_list[i] == action:
            data_x[counter] = cov[i,:]
            data_y[counter] = labels[i,action]
            counter = counter + 1
    return (data_x,data_y) 


def select_action(values,params):
    (mu,gamma) = params
    k = mu
    max_value = np.amax(values)
    max_index = np.where(values == max_value)[0][0]
    prob = np.zeros(len(values))
    for i in range(k): 
        if i == max_index:
            next
        else: 
            prob[i] = 1./(mu + gamma*(max_value - values[i]))
    prob[max_index] = 1 - np.sum(prob)
    prob = prob/np.sum(prob)
    #print('probability')
    #print(prob)
    #TODO roulette wheel
    draw = np.random.rand()
    sums = 0
    action = 0
    for i in range(k):
        sums = sums + prob[i]
        if sums >= draw:
            action = i
            break
    return action

mean_reward = contextual_bandit(data)

In [ ]:
import matplotlib.pyplot as plt
domain = range(len(mean_reward))
plt.scatter(domain, mean_reward)
plt.show()

In [ ]:
a = np.array([[2,3],[1,4]])
b = np.sort(a,axis=0)
print(b)

In [ ]:
a = np.array([3,2,1])
order = a.argsort()
print(order)
b = a[order]
print(b)


In [ ]:
a = np.zeros((2,3))
a.shape[1]

In [ ]:
x = tf.constant([0.5,0.5,0.5])
y = tf.constant([1.,0.,1.])
weights = tf.constant([0.4,0.2,0.4])
t = tf.ones(3)


arg1 = tf.multiply(y,tf.math.log(x))
arg2 = tf.multiply(tf.subtract(t,y),tf.math.log(tf.math.subtract(t,x)))
q = tf.reduce_sum(tf.multiply(tf.add(arg1,arg2),weights))
print(q)

print(tf.math.scalar_mul(-1.0,x))

In [ ]:
print(x)